In [1]:
#Importações

import os
import sys
import pandas as pd
import numpy as np
sys.path.append(os.path.abspath('..'))
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from functions.analise_exploratoria import (load_and_combine_csvs,
                                            clean_dataframe,
                                            add_confidential_flags,
                                            _estimate_state_row,
                                            apply_state_estimation)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [2]:
csv_path = '../raw_data'
df_raw = load_and_combine_csvs(csv_path)

In [3]:
df_raw.columns

Index(['CÓDIGO ÓRGÃO SUPERIOR', 'NOME ÓRGÃO SUPERIOR', 'CÓDIGO ÓRGÃO',
       'NOME ÓRGÃO', 'CÓDIGO UNIDADE GESTORA', 'NOME UNIDADE GESTORA',
       'ANO EXTRATO', 'MÊS EXTRATO', 'CPF PORTADOR', 'NOME PORTADOR',
       'CNPJ OU CPF FAVORECIDO', 'NOME FAVORECIDO', 'TRANSAÇÃO',
       'DATA TRANSAÇÃO', 'VALOR TRANSAÇÃO', 'ARQUIVO_ORIGEM'],
      dtype='object')

In [4]:
df_limpo = clean_dataframe(df_raw)

In [5]:
df_conf = add_confidential_flags(df_limpo)

In [6]:
df_final = apply_state_estimation(df_conf)

In [7]:
df_final.dtypes

CÓDIGO ÓRGÃO SUPERIOR              int64
NOME ÓRGÃO SUPERIOR               object
CÓDIGO ÓRGÃO                       int64
NOME ÓRGÃO                        object
CÓDIGO UNIDADE GESTORA             int64
NOME UNIDADE GESTORA              object
ANO EXTRATO                        int64
MÊS EXTRATO                        int64
CPF PORTADOR                      object
NOME PORTADOR                     object
CNPJ OU CPF FAVORECIDO             int64
NOME FAVORECIDO                   object
TRANSAÇÃO                         object
DATA TRANSAÇÃO            datetime64[ns]
VALOR TRANSAÇÃO                  float64
ARQUIVO_ORIGEM                    object
SIGILOSO                           int64
ESTADO_ESTIMADO                   object
dtype: object

In [8]:
# Example checks
print(df_final.info())
print(df_final.head())

<class 'pandas.core.frame.DataFrame'>
Index: 307307 entries, 0 to 392088
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   CÓDIGO ÓRGÃO SUPERIOR   307307 non-null  int64         
 1   NOME ÓRGÃO SUPERIOR     307307 non-null  object        
 2   CÓDIGO ÓRGÃO            307307 non-null  int64         
 3   NOME ÓRGÃO              307307 non-null  object        
 4   CÓDIGO UNIDADE GESTORA  307307 non-null  int64         
 5   NOME UNIDADE GESTORA    307307 non-null  object        
 6   ANO EXTRATO             307307 non-null  int64         
 7   MÊS EXTRATO             307307 non-null  int64         
 8   CPF PORTADOR            289417 non-null  object        
 9   NOME PORTADOR           307307 non-null  object        
 10  CNPJ OU CPF FAVORECIDO  307307 non-null  int64         
 11  NOME FAVORECIDO         307307 non-null  object        
 12  TRANSAÇÃO               307307 non-

In [9]:
df_final.head()

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,ANO EXTRATO,MÊS EXTRATO,CPF PORTADOR,NOME PORTADOR,CNPJ OU CPF FAVORECIDO,NOME FAVORECIDO,TRANSAÇÃO,DATA TRANSAÇÃO,VALOR TRANSAÇÃO,ARQUIVO_ORIGEM,SIGILOSO,ESTADO_ESTIMADO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110161,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,2024,3,***.725.752-**,VIVIANE CORREA LIMA,2179328000142,FREITAS & CIA LTDA,COMPRA A/V - R$ - APRES,2024-02-21,8.42,202403_CPGF.csv,0,UNIÃO
1,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110161,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,2024,3,***.866.951-**,JONAS SCHOTTZ DA SILVA,37828985000158,CASA DO SINDICO LTDA,COMPRA A/V - R$ - APRES,2024-02-20,72.00,202403_CPGF.csv,0,UNIÃO
2,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110161,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,2024,3,***.866.951-**,JONAS SCHOTTZ DA SILVA,33216995000181,RADEL ELETRONICA LTDA,COMPRA A/V - R$ - APRES,2024-02-21,26.00,202403_CPGF.csv,0,UNIÃO
3,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110161,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,2024,3,***.866.951-**,JONAS SCHOTTZ DA SILVA,10421458000178,T-9 ELETRONICA E INFORMATICA LTDA,COMPRA A/V - R$ - APRES,2024-02-07,1352.00,202403_CPGF.csv,0,UNIÃO
4,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110161,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,2024,3,***.725.752-**,VIVIANE CORREA LIMA,16577772000120,SO FILTROS RONDONIA LTDA,COMPRA A/V - R$ - APRES,2024-02-21,570.00,202403_CPGF.csv,0,UNIÃO


In [10]:
df_final['SIGILOSO'].value_counts()

SIGILOSO
0    289417
1     17890
Name: count, dtype: int64

In [ ]:
# Identifying columns
cat_columns = [
    'NOME ÓRGÃO SUPERIOR', 'NOME ÓRGÃO', 'NOME UNIDADE GESTORA', 'CPF PORTADOR',
    'NOME PORTADOR', 'NOME FAVORECIDO', 'TRANSAÇÃO', 'ARQUIVO_ORIGEM', 'ESTADO_ESTIMADO',
    'ID_PORTADOR'
]
# Numerical columns
# Columns for median imputation
num_columns = [
    'CÓDIGO ÓRGÃO SUPERIOR', 'CÓDIGO ÓRGÃO', 'CÓDIGO UNIDADE GESTORA', 'ANO EXTRATO',
    'MÊS EXTRATO', 'CNPJ OU CPF FAVORECIDO', 'VALOR TRANSAÇÃO', 'SIGILOSO', 'FIM_DE_SEMANA'
]

# Optionally add more engineered features later

In [ ]:
# Numerical pipeline: impute and scale
num_pipeline = Pipeline([
 #   ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

# Categorical pipeline: impute and encode
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Desconhecido')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine in a column transformer
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_columns),
    ('cat', cat_pipeline, cat_columns)
])